In [6]:
import torch

In [7]:
import os
from PIL import Image
import glob

from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms

In [8]:
class myDataset(Dataset):

    def read_dataset(self):

        images_path = self.images_path
        masks_path = self.masks_path
        imgs =  glob.glob(images_path + "/*.jpg")
        masks = glob.glob(masks_path + "/*.png")

        return imgs, masks, len(imgs)

    def __init__(self, images_path,masks_path, transforms=None):
        self.images_path = images_path
        self.masks_path= masks_path
        self.images, self.masks, self.length= self.read_dataset()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.images[index])
        image = image.convert("RGB")
        
        mask = Image.open(self.masks[index])
        mask = image.convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image)
            mask = self.transforms(mask)

        return image, mask

    def __len__(self):
        return self.length

In [9]:
transforms_train = transforms.Compose([transforms.Resize((256, 256)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])

transforms_valid = transforms.Compose([transforms.Resize((256, 256)),
                                       transforms.ToTensor()])

In [10]:
train_dataset = myDataset(cfg.TRAIN_IMAGE_PATH, cfg.TRAIN_ANNOT_DIR, transforms_train)
trainloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

valid_dataset = myDataset(cfg.VAL_IMAGE_PATH,cfg.VAL_ANNOT_DIR,transforms=transforms_valid)
validloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)

In [12]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [13]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits



In [14]:
model = UNet (3,3)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [22]:
import torch.optim as optim
import torch.nn.functional as F

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [18]:
loss_list = []
accuracy_list = []

In [ ]:
for epoch in range (20):
    
    running_loss = 0.0
    nb_data = 0.
    for i, data in enumerate(trainloader, 0):

        inputs, masks = data
        optimizer.zero_grad()
        inputs, masks = inputs.to(device), masks.to(device)

        outputs = model(inputs)             
     
        loss = criterion(outputs, masks)
        loss.backward()                    
        optimizer.step()                   
 

        running_loss += loss.item()
        
        nb_data += 1.
        
    running_loss = running_loss / nb_data
    loss_list.append(running_loss)

   #  correct = 0.
   #  total = 0.
   #  with torch.no_grad():
   #      for data in validloader:
   #          images, labels = data
   #          images, labels = images.to(device), labels.to(device)
   #          outputs = model(images)
   #          loss = criterion(outputs, images)
 
    
    print("Epoch ", epoch, "; train loss = ", running_loss
          
          )